In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import DataBaseManager,add_merged_contours_lines_multiprocess
import mysql.connector


In [2]:
#set global net_buffer_length=1000000; 
#set global max_allowed_packet=1000000000;

In [3]:

user='spirz'
password='this_is_my_PASSWORD_m8'
host='localhost'
database='dem_from_contours_lines'
contours_lines_table_name='contours_lines'
osm_nodes_table_name="osm_nodes"
osm_edges_table_name="osm_edges"
intersections_table_name="intersections"
2

2

In [4]:
file_paths=glob.glob('contours_lines_corse/*.shp')
elevation_column="ALTITUDE"

In [5]:
len(file_paths)

11

In [6]:
DBM=DataBaseManager(user=user,password=password,host=host,database=database,
            contours_lines_table_name=contours_lines_table_name,
            osm_nodes_table_name=osm_nodes_table_name,
            osm_edges_table_name=osm_edges_table_name,
            intersections_table_name=intersections_table_name)
2

2

In [8]:
DBM.execute("SELECT id FROM %s WHERE NOT is_closed"%contours_lines_table_name)
len(DBM.cursor.fetchall())

6905

51410 closed
53778

In [9]:
DELETE_TABLES=True
if DELETE_TABLES:
    t1=time.time()
    DBM.delete_tables()
    t2=time.time()
    print(t2-t1)

0.28765249252319336


In [10]:
CREATE_TABLES=True
if CREATE_TABLES:
    t1=time.time()
    DBM.create_tables_v1()
    t2=time.time()
    print(t2-t1)    



0.06378173828125


In [ ]:
#ASSUME GEODATAFRAMES ARE IN UTM COORDINATES
GENERATE=True
if GENERATE:
    t1=time.time()
    DBM.add_contour_lines_to_database(file_paths,elevation_column=elevation_column)
    t2=time.time()
    print(t2-t1)
2

file  1/11
reading file took 12.844422578811646
preprocess df took 3.225516
inserting 10158 rows took 17.987822
comparing 0 pairs took 0.664915
file  2/11
reading file took 18.549926280975342
preprocess df took 4.706886


In [ ]:
700.4467239379883

In [10]:
t1=time.time()
add_merged_contours_lines_multiprocess(nb_cpu=None)
t2=time.time()
print(t2-t1)

7 cpus working
elevation 10/77
elevation 10/77
elevation 10/77
elevation 10/77
elevation 10/78
elevation 10/77
elevation 10/77
elevation 20/77elevation 20/77

elevation 20/77
elevation 20/77
elevation 20/77
elevation 20/78
elevation 20/77
elevation 30/77elevation 30/77

elevation 30/77
elevation 30/77
elevation 30/77
elevation 30/77
elevation 30/78
elevation 40/77
elevation 40/77
elevation 40/77
elevation 40/77
elevation 40/77
elevation 40/78
elevation 40/77
elevation 50/77
elevation 50/77
elevation 50/77
elevation 50/77
elevation 50/77
elevation 50/78
elevation 50/77
elevation 60/77
elevation 60/77
elevation 60/77
elevation 60/77
elevation 60/77
elevation 60/77
elevation 60/78
elevation 70/77
elevation 70/77
elevation 70/77
elevation 70/77
elevation 70/77
elevation 70/77
elevation 70/78
1053.968792438507


In [ ]:
MERGE=True
if MERGE:
    t1=time.time()
    DBM.add_merged_contours_lines()
    t2=time.time()
    print(t2-t1)
2